In [8]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from tensorflow.examples.tutorials.mnist import input_data

In [6]:
mnist = input_data.read_data_sets("MNIST_data/")
x_train = mnist.train.images.reshape([-1, 28, 28, 1]) # (m, h, w, c)
y_train = mnist.train.labels
x_test = mnist.test.images.reshape([-1, 28, 28, 1])
y_test = mnist.test.labels
x_train.shape, x_test.shape

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


((55000, 28, 28, 1), (10000, 28, 28, 1))

레이어와 레이어 사이에는 반드시 비선형 함수가 존재해야 합니다.
sigmoid : 비선형함수
tf.nn.relu : max(0, x) : 비선형함수
y = ax  
y = (relu(ax)) * b # ax : 히든레이어
y = (sigmoid(ax * b)) * c # ax * b : 2번째 히든레이어
y = (a*b*c) * x ... 와 같이 비선형 함수가 없다면 선형함수를 여러번 적용하는 것이 의미가 없습니다

In [ ]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.int32, [None])
net = slim.convolution2d(X, num_outputs=16, activation_fn=tf.nn.relu, kernel_size=[3,3])
net = slim.max_pool2d(net, kernel_size=[2,2])
net = slim.convolution2d(net, num_outputs=32, kernel_size=[3,3])
net = slim.max_pool2d(net, kernel_size=[2,2])#(?, 7, 7, 32)
net = slim.flatten(net)#(?, 1568)
net = slim.fully_connected(net, num_outputs=16)
z = slim.fully_connected(net, num_outputs=10)#(?, 10) # 점수, 가능성
h = tf.nn.softmax(z)#점수를 확률로 변환 
cost = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=z)
cost = tf.reduce_mean(cost)
train = tf.train.GradientDescentOptimizer(0.5).minimize(cost)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [27]:
for i in range(5): #모델이 커졌으면 반복횟수도 늘려야 합니다.
    _, _cost = sess.run([train, cost], {X:x_train, Y:y_train})
    print(i, _cost)
p = np.argmax( sess.run(h, {X:x_train} ) , axis=-1)
_cost, accuracy_score(y_train, p)

0 2.3158543
1 2.2854419
2 2.2713935
3 2.2516112
4 2.221166


(2.221166, 0.2334)

In [29]:
p = np.argmax( sess.run(h, {X:x_test} ) , axis=-1)
accuracy_score(y_test, p)

0.2264

In [30]:
print(classification_report(y_test, p))

              precision    recall  f1-score   support

           0       0.15      1.00      0.27       980
           1       0.99      0.47      0.64      1135
           2       0.00      0.00      0.00      1032
           3       0.88      0.05      0.09      1010
           4       0.00      0.00      0.00       982
           5       0.00      0.00      0.00       892
           6       0.23      0.73      0.35       958
           7       0.00      0.00      0.00      1028
           8       0.00      0.00      0.00       974
           9       0.00      0.00      0.00      1009

   micro avg       0.23      0.23      0.23     10000
   macro avg       0.23      0.23      0.14     10000
weighted avg       0.24      0.23      0.14     10000



C:\Users\User\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
